# Metaclasses
### A quick overview

A high level explanation is necessary before we get down to the details.

A class is an object, and just like any other object, it's an instance of something: a metaclass. The default metaclass is type. Unfortunately, due to backwards compatibility, type is a bit confusing: it can also be used as a function that return the class of an object

In [1]:
class Foobar:
     pass

type(Foobar)

type

In [2]:
foo = Foobar()
type(foo)

__main__.Foobar

In [3]:
isinstance(foo, Foobar)

True

In [4]:
isinstance(Foobar, type)

True

To put this in picture:  
![pic](instance-of.png)

### Simple metaclass use

We can use type directly to make a class, without any class statement:

In [5]:
MyClass = type('MyClass', (), {})
MyClass

__main__.MyClass

The class statement isn't just syntactic sugar, it does some extra things, like setting an adequate `__qualname__` and `__doc__` properties or calling `__prepare__`.

We can make a custom metaclass:

In [7]:
class Meta(type):
    pass

In [8]:
class Complex(metaclass=Meta):
    pass

type(Complex)

__main__.Meta

### Magic methods
One distinctive feature of Python is magic methods: they allow the programmer to override behavior for various operators and behavior of objects. To override the call operator you'd do this:

In [ ]:
class Funky:
     def __call__(self):
         print("Look at me, I work like a function!")

f = Funky()
f()

Metaclasses rely on several magic methods so it's quite useful to know a bit more about them.
### The slots

When you define a magic method in your class the function will end up as a pointer in a [struct that describes the class](https://docs.python.org/3/c-api/typeobj.html), in addition to the entry in `__dict__`. That struct [7] has a field for each magic method. For some reason these fields are called type slots.

Now there's another feature, implemented via the `__slots__` [attribute](https://docs.python.org/3/reference/datamodel.html#object.__slots__). A class with `__slots__` will create instances that don't have a `__dict__` (they use a little bit less memory). A side-effect of this is that instances cannot have other fields than what was specified in `__slots__`: if you try to set an unexpected field you'll get an exception.

For the scope of this article when slots are mentioned it will mean the type slots, not `__slots__`.

### Object attribute lookup
Now this is something that's easy to get wrong because of the many slight differences to old-style objects in Python 2.

Assuming Class is the class and instance is an instance of Class, evaluating instance.foobar roughly equates to this:

- Call the type slot for Class.`__getattribute__` (tp_getattro). The default does this: [10]
  - Does Class.`__dict__` have a foobar item that is a data descriptor [8]?
     - If yes, return the result of Class.`__dict__`['foobar'].`__get__`(instance, Class). [6]
  - Does instance.`__dict__` have a foobar item in it?
     - If yes, return instance.`__dict__`['foobar'].
  - Does Class.`__dict__` have a foobar item that is not a data descriptor [9]?
     - If yes, return the result of Class.`__dict__`['foobar'].`__get__`(instance, klass). [6]
  - Does Class.`__dict__` have a foobar item?
     - If yes, return the result of Class.`__dict__`['foobar'].
- If the attribute still wasn't found, and there's a Class.`__getattr__`, call Class.`__getattr__`('foobar').

Still not clear? Perhaps a diagram normal attribute lookup helps:

![Lookup](lookup.png "Title")

```
To avoid creating confusions with the “.” operator doing crazy things I've used “:” in this diagram to signify the location.
```

### Class attribute lookup

Because classes needs to be able support the classmethod and staticmethod properties [6] when you evaluate something like Class.foobar the lookup is slightly different than what would happen when you evaluate instance.foobar.

Assuming Class is an instance of Metaclass, evaluating Class.foobar roughly equates to this:

- Call the type slot for Metaclass.`__getattribute__` (tp_getattro). The default does this: [11]
  - Does Metaclass.`__dict__` have a foobar item that is a data descriptor [8]?
     - If yes, return the result of Metaclass.`__dict__`['foobar'].`__get__`(Class, Metaclass). [6]
  - Does Class.`__dict__` have a foobar item that is a descriptor (of any kind)?
     - If yes, return the result of Class.`__dict__`['foobar'].`__get__`(None, Class). [6]
  - Does Class.`__dict__` have a foobar item in it?
     - If yes, return Class.`__dict__`['foobar'].
  - Does Metaclass.`__dict__` have a foobar item that is not a data descriptor [9]?
     - If yes, return the result of Metaclass.`__dict__`['foobar'].`__get__`(Class, Metaclass). [6]
  - Does Metaclass.`__dict__` have any foobar item?
     - If yes, return Metaclass.`__dict__`['foobar'].
- If the attribute still wasn't found, and there's a Metaclass.`__getattr__`, call Metaclass.`__getattr__`('foobar').

The whole shebang would look like this in a diagram:

![Lookup-2](class-attribute-lookup.png "Title")

### The `__new__` method

One of the most common point of confusion with both classes and metaclasses is the `__new__` method. It has some very special conventions.

The `__new__` method is the constructor (it returns the new instance) while `__init__` is just a initializer (the instance is already created when `__init__` is called).

Suppose have a class like this:

```
class Foobar:
    def __new__(cls):
        return super().__new__(cls)
```

Now if you recall the previous section, you'd expect that `__new__` would be looked up on the metaclass, but alas, it wouldn't be so useful that way [19] so it's looked up statically.

When the Foobar class wants this magic method it will be looked up on the same object (the class), not on a upper level like all the other magic methods. This is very important to understand, because both the class and the metaclass can define this method:

- `Foobar.__new__` is used to create instances of Foobar
- `type.__new__` is used to create the Foobar class (an instance of type in the example)

### The `__prepare__` method

This method is called before the class body is executed and it must return a dictionary-like object that's used as the local namespace for all the code from the class body. It was added in Python 3.0, see PEP-3115.

If your `__prepare__` returns an object x then this:

```
class Class(metaclass=Meta):
    a = 1
    b = 2
    c = 3
```

Will make the following changes to x:

```
x['a'] = 1
x['b'] = 2
x['c'] = 3
```

This x object needs to look like a dictionary. Note that this x object will end up as an argument to Metaclass.`__new__` and if it's not an instance of dict you need to convert it before calling super().`__new__`.

Interestingly enough this method doesn't have `__new__`'s special lookup. It appears it doesn't have it's own type slot and it's looked up via the class attribute lookup, if you read back a bit. 

### Putting it all together

To start things off, a diagram of how instances are constructed:

![Lookup-3](instance-creation.png "Title")

How to read this swim lane diagram:

The horizontal lanes is the place where you define the functions.

- Solid lines mean a function call.
- A line from Metaclass.`__call__` to Class.`__new__` means Metaclass.`__call__` will call Class.`__new__`.
  - Dashed lines means something is returned.
- Class.`__new__` returns the instance of Class.
  - Metaclass.`__call__` returns whatever Class.`__new__` returned (and if it returned an instance of Class it will also call Class.`__init__` on it). [16]
  - The number in the red circle signifies the call order.
  
- Creating a class is quite similar:  

![Lookup-4](class-creation.png "Title")

Few more notes:

- Metaclass.`__prepare__` just returns the namespace object (a dictionary-like object as explained before).
- Metaclass.`__new__` returns the Class object.
- MetaMetaclass.`__call__` returns whatever Metaclass.`__new__` returned (and if it returned an instance of Metaclass it will also call Metaclass.`__init__` on it). [16]

So you see, metaclasses allow you to customize almost every part of an object life-cycle.

### Metaclasses are callables

Metaclasses are callables
If you look again at the diagrams, you'll notice that making an instance goes through Metaclass.__call__. This means you can use any callable as the metaclass:


In [11]:
class Foo(metaclass=print):  # pointless, but illustrative
    pass

print(Foo)

Foo () {'__module__': '__main__', '__qualname__': 'Foo'}
None


### Subclasses inherit the metaclass

One advantage compared to class decorators is the fact that subclasses inherit the metaclass.

This is a consequence of the fact that Metaclass(...) returns an object which usually has Metaclass as the `__class__`.

### The method signatures

There are still few important details missing, like the method signatures. Lets look at class and metaclass with all the important stuff implemented.

Note the extra **kwargs - those are the extra keywords arguments you can pass in the class statement.

```
class Meta(type):
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print('  Meta.__prepare__(mcs=%s, name=%r, bases=%s, **%s)' % (
            mcs, name, bases, kwargs
        ))
        return {}
```

As mentioned before, `__prepare__` can return objects that are not dict instances, so you need to make sure your `__new__` handles that.

```
    def __new__(mcs, name, bases, attrs, **kwargs):
         print('  Meta.__new__(mcs=%s, name=%r, bases=%s, attrs=[%s], **%s)' % (
             mcs, name, bases, ', '.join(attrs), kwargs
         ))
         return super().__new__(mcs, name, bases, attrs)
```

It's uncommon to see `__init__` being implemented in a metaclass because it's not that powerful - the class is already constructed when `__init__` is called. It roughly equates to having a class decorator with the difference that `__init__` would get run when making subclasses, while class decorators are not called for subclasses.

```
     def __init__(cls, name, bases, attrs, **kwargs):
        print('  Meta.__init__(cls=%s, name=%r, bases=%s, attrs=[%s], **%s)' % (
        cls, name, bases, ', '.join(attrs), kwargs
        ))
        return super().__init__(name, bases, attrs)
```

The `__call__` method will be called when you make instances of Class.

```
     def __call__(cls, *args, **kwargs):
         print('  Meta.__call__(cls=%s, args=%s, kwargs=%s)' % (
             cls, args, kwargs
         ))
         return super().__call__(*args, **kwargs)
...
```

Using Meta, note the extra=1


In [13]:
class Meta(type):
    @classmethod
    def __prepare__(mcs, name, bases, **kwargs):
        print('  Meta.__prepare__(mcs=%s, name=%r, bases=%s, **%s)' % (
            mcs, name, bases, kwargs
        ))
        return {}
        
    def __new__(mcs, name, bases, attrs, **kwargs):
        print('  Meta.__new__(mcs=%s, name=%r, bases=%s, attrs=[%s], **%s)' % (
            mcs, name, bases, ', '.join(attrs), kwargs
        ))
        return super().__new__(mcs, name, bases, attrs)
     
    def __init__(cls, name, bases, attrs, **kwargs):
        print('  Meta.__init__(cls=%s, name=%r, bases=%s, attrs=[%s], **%s)' % (
            cls, name, bases, ', '.join(attrs), kwargs
        ))
        super().__init__(name, bases, attrs)
      
    def __call__(cls, *args, **kwargs):
        print('  Meta.__call__(cls=%s, args=%s, kwargs=%s)' % (
           cls, args, kwargs
        ))
        return super().__call__(*args, **kwargs)

In [14]:
class Class(metaclass=Meta, extra=1):
    def __new__(cls, myarg):
        print('  Class.__new__(cls=%s, myarg=%s)' % (
            cls, myarg
        ))
        return super().__new__(cls)

    def __init__(self, myarg):
        print('  Class.__init__(self=%s, myarg=%s)' % (
            self, myarg
        ))
        self.myarg = myarg
        super().__init__()

    def __str__(self):
        return "<instance of Class; myargs=%s>" % (
            getattr(self, 'myarg', 'MISSING'),
        )

  Meta.__prepare__(mcs=<class '__main__.Meta'>, name='Class', bases=(), **{'extra': 1})
  Meta.__new__(mcs=<class '__main__.Meta'>, name='Class', bases=(), attrs=[__module__, __qualname__, __new__, __init__, __str__, __classcell__], **{'extra': 1})
  Meta.__init__(cls=<class '__main__.Class'>, name='Class', bases=(), attrs=[__module__, __qualname__, __new__, __init__, __str__, __classcell__], **{'extra': 1})


In [15]:
Class(1)


  Meta.__call__(cls=<class '__main__.Class'>, args=(1,), kwargs={})
  Class.__new__(cls=<class '__main__.Class'>, myarg=1)
  Class.__init__(self=<instance of Class; myargs=MISSING>, myarg=1)
